## Geleneksel makine öğrenme metotları

In [1]:
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

### AG News: Haber metinlerinin sınıflandırılması.

In [2]:
# AG News veri setini yükleme
dataset = load_dataset('ag_news')

# Eğitim veri setinden örnekleme (10.000 örnek)
train_data = pd.DataFrame(dataset['train'])
train_data_sampled = train_data.sample(10000, random_state=42)

# Test veri seti
test_data = pd.DataFrame(dataset['test'])

# Metin ve etiketleri ayırma
X_train_texts = train_data_sampled['text']
y_train = train_data_sampled['label']
X_test_texts = test_data['text']
y_test = test_data['label']

Found cached dataset parquet (C:/Users/onura/.cache/huggingface/datasets/parquet/ag_news-f4012edcb412d6fa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Veri seti bilgilerini yazdırma
for split in dataset.keys():
    print(f"--- {split.upper()} ---")
    print(f"Örnek sayısı: {len(dataset[split])}")
    
    # Ortalama metin uzunluğu
    total_text_length = sum(len(item['text']) for item in dataset[split])
    avg_text_length = total_text_length / len(dataset[split])
    print(f"Toplam metin uzunluğu: {total_text_length}")
    print(f"Ortalama metin uzunluğu: {avg_text_length:.2f} karakter")
    
    print()

--- TRAIN ---
Örnek sayısı: 120000
Toplam metin uzunluğu: 28377303
Ortalama metin uzunluğu: 236.48 karakter

--- TEST ---
Örnek sayısı: 7600
Toplam metin uzunluğu: 1788274
Ortalama metin uzunluğu: 235.30 karakter



### Embedding yöntemi olarak TF-IDF kullanılmıştır.

In [4]:
# TF-IDF ile metinleri vektörleştirme (özellik sayısı 1000 ile sınırlandırıldı)
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train = vectorizer.fit_transform(X_train_texts)
X_test = vectorizer.transform(X_test_texts)

### Geleneksel Makine Öğrenme Metotları

In [5]:
# Modellerin tanımlanması
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=10, n_jobs=-1)  # Paralelleştirme
}

results = []

# Her model için eğitim ve değerlendirme
for name, model in models.items():
    # Modeli eğit
    model.fit(X_train, y_train)
    
    # Test verisi üzerinde tahmin yap
    y_pred = model.predict(X_test)
    
    # Performans metriklerini hesapla
    results.append({
        "Model": name,
        "Precision": precision_score(y_test, y_pred, average='weighted'),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "Accuracy": accuracy_score(y_test, y_pred),
        "F-Score": f1_score(y_test, y_pred, average='weighted')
    })

# Sonuçları tablo olarak göster
results_df = pd.DataFrame(results)
print(results_df)

                 Model  Precision    Recall  Accuracy   F-Score
0  Logistic Regression   0.847948  0.848684  0.848684  0.848192
1                  SVM   0.855292  0.856053  0.856053  0.855485
2                  KNN   0.729756  0.431711  0.431711  0.410235
3        Random Forest   0.791818  0.792500  0.792500  0.791186


-----------
## Derin Öğrenme Metotları

In [7]:
import gensim
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, LSTM, Dropout
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

#### Word2Vec ile CNN ve LSTM Modelleri:

In [8]:
# Eğitim verisini kelime listelerine ayırma
X_train_tokens = [sentence.split() for sentence in X_train_texts]
X_test_tokens = [sentence.split() for sentence in X_test_texts]

# Word2Vec modeli eğitme
word2vec_model = Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=1, workers=4)

# Kelime vektörlerini içeren bir sözlük
word_vectors = word2vec_model.wv
vocab_size = len(word_vectors)
print(f"Kelime vektörlerinin boyutu: {word_vectors.vector_size}")

Kelime vektörlerinin boyutu: 100


#### Embedding Matrisini Hazırlama:

In [9]:
# Embedding matrisini oluşturma
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))  # +1, bilinmeyen kelimeler için
word_index = {word: idx + 1 for idx, word in enumerate(word_vectors.index_to_key)}

for word, idx in word_index.items():
    embedding_matrix[idx] = word_vectors[word]

#### Metinleri sayısal formata çevirme

In [10]:
# Kelimeleri dizilere dönüştürme
def text_to_sequence(tokens, word_index):
    return [[word_index.get(word, 0) for word in sentence] for sentence in tokens]

X_train_seq = text_to_sequence(X_train_tokens, word_index)
X_test_seq = text_to_sequence(X_test_tokens, word_index)

# Dizileri aynı uzunluğa getirerek doldurma
max_length = 100
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

### CNN Modeli

In [11]:
# CNN modeli
cnn_model = Sequential([
    Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 sınıf
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

# CNN modelini eğitme
cnn_model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_data=(X_test_padded, y_test))

C:\Users\onura\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     4,815,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,815,300 (18.37 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,815,300 (18.37 MB)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.2883 - loss: 1.4742 - val_accuracy: 0.3480 - val_loss: 1.3470
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3197 - loss: 1.3459 - val_accuracy: 0.4305 - val_loss: 1.2745
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3589 - loss: 1.3152 - val_accuracy: 0.4026 - val_loss: 1.2659
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3596 - loss: 1.3029 - val_accuracy: 0.4471 - val_loss: 1.2424
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3973 - loss: 1.2723 - val_accuracy: 0.4533 - val_loss: 1.2140


### LSTM Modeli

In [12]:
# LSTM modeli
lstm_model = Sequential([
    Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 sınıf
])

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lstm_model.summary()

# LSTM modelini eğitme
lstm_model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_data=(X_test_padded, y_test))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │     4,815,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,815,300 (18.37 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,815,300 (18.37 MB)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 25s 133ms/step - accuracy: 0.2459 - loss: 1.3864 - val_accuracy: 0.2503 - val_loss: 1.3844
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 126ms/step - accuracy: 0.2716 - loss: 1.3836 - val_accuracy: 0.2709 - val_loss: 1.3803
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 127ms/step - accuracy: 0.2649 - loss: 1.3848 - val_accuracy: 0.2770 - val_loss: 1.3808
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 126ms/step - accuracy: 0.2543 - loss: 1.3846 - val_accuracy: 0.2680 - val_loss: 1.3829
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 124ms/step - accuracy: 0.2683 - loss: 1.3835 - val_accuracy: 0.2926 - val_loss: 1.3791


### Performans değerlendirme

In [13]:
# CNN performans değerlendirme
cnn_preds = np.argmax(cnn_model.predict(X_test_padded), axis=1)
cnn_precision = precision_score(y_test, cnn_preds, average='weighted')
cnn_recall = recall_score(y_test, cnn_preds, average='weighted')
cnn_accuracy = accuracy_score(y_test, cnn_preds)
cnn_f1 = f1_score(y_test, cnn_preds, average='weighted')

print(f"CNN -> Precision: {cnn_precision}, Recall: {cnn_recall}, Accuracy: {cnn_accuracy}, F-Score: {cnn_f1}")

# LSTM performans değerlendirme
lstm_preds = np.argmax(lstm_model.predict(X_test_padded), axis=1)
lstm_precision = precision_score(y_test, lstm_preds, average='weighted')
lstm_recall = recall_score(y_test, lstm_preds, average='weighted')
lstm_accuracy = accuracy_score(y_test, lstm_preds)
lstm_f1 = f1_score(y_test, lstm_preds, average='weighted')

print(f"LSTM -> Precision: {lstm_precision}, Recall: {lstm_recall}, Accuracy: {lstm_accuracy}, F-Score: {lstm_f1}")

238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
CNN -> Precision: 0.5009452272668855, Recall: 0.4532894736842105, Accuracy: 0.4532894736842105, F-Score: 0.43890446054200866
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step
LSTM -> Precision: 0.3199368824024042, Recall: 0.2926315789473684, Accuracy: 0.2926315789473684, F-Score: 0.24735664586309092
